# Version 2.0 - removal of the 8, 9 and 10 labels (No face, uncertain or unclear), using 64, 128, 256, 512 nodes for layers, Linear activation for the regression problem, RELU intermediate function.

Current series of approaches is directed towards transforming the problem from a classification one into the numerical regression one. Considering that the original Affect-Net dataset has variables ```valence``` and ```arousal``` defining emotional state of the person in numerical way it is possible to perform regression and then, considering found numerical values to make a classification out of given numerical values.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers

import gc

from point_net_api import *

tf.random.set_seed(1234)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5547148179488211095
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750390784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10549531820485899051
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
1 Physical GPUs, 1 Logical GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



## Reading the data

In [3]:
train_df = read_all_pickles(package_name='pkls_valence_arousal', file_specification="_landmarks_valence_arousal.pkl")
print('original length = ' + str(len(train_df)))

#   drop empty records
train_df.dropna(inplace=True)

#   ignoring the last three labels because they are not providing necessary information
train_df = train_df[train_df.emotion != 8]
train_df = train_df[train_df.emotion != 9]
train_df = train_df[train_df.emotion != 10]
train_df.drop(columns=['emotion'], inplace=True)

print('updated length = ' + str(len(train_df)))

train_df.head(10)

original length = 409797
updated length = 272340


,path,valence,arousal,landmarks
400002,Manually_Annotated_Images/325/f489e8169ea30897...,0.577985,0.271739,"[[0.5394599437713623, 0.676323413848877, -0.08..."
400003,Manually_Annotated_Images/264/224ee36d52b497b1...,-0.384534,0.110619,"[[0.5218021273612976, 0.7440579533576965, -0.1..."
400005,Manually_Annotated_Images/980/8262220f6a3bcfd7...,0.639259,0.034451,"[[0.5117629766464233, 0.7039216160774231, -0.0..."
400007,Manually_Annotated_Images/609/5b3d3b8b9d89deaf...,0.705858,-0.258112,"[[0.47549158334732056, 0.7176483273506165, -0...."
400009,Manually_Annotated_Images/958/7b235cdca2ca0430...,-0.103175,-0.341270,"[[0.5392422080039978, 0.7580064535140991, -0.0..."
400013,Manually_Annotated_Images/715/bff238e798b70ccf...,-0.595238,0.428571,"[[0.49899062514305115, 0.7014129161834717, -0...."
400014,Manually_Annotated_Images/514/f92fad99d07e4b9e...,-0.101626,0.895277,"[[0.47227370738983154, 0.6721517443656921, -0...."
400015,Manually_Annotated_Images/728/9629f4764d5956d9...,0.526225,-0.081953,"[[0.519411563873291, 0.739055871963501, -0.060..."
400016,Manually_Annotated_Images/866/ec55abeb8a12d104...,0.871080,0.183895,"[[0.5619162917137146, 0.6456629633903503, -0.0..."
400017,Manually_Annotated_Images/907/7afaf57f140a6270...,0.691384,-0.062657,"[[0.5264552235603333, 0.7333130240440369, -0.0..."


## Split the data into the training and validation sets

In [4]:
X = np.array(list(map(np.float64, np.array(train_df['landmarks']))))
Y = np.array(train_df[['valence', 'arousal']])

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
del X
del Y
del train_df

gc.collect()

22

## Setting NN

In [6]:
first_degree, second_degree, third_degree, fourth_degree = 64, 128, 256, 512

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 32)
x = conv_bn(x, first_degree)
x = conv_bn(x, third_degree)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.4)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(2, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 468, 32)      0           ['batch_normalization

## Forming TF slices

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(len(x_train)).batch(32)
test_dataset = test_dataset.shuffle(len(x_test)).batch(32)

In [8]:
del x_train
del y_train
del x_test
del y_test

gc.collect()

211

## Train model

In [18]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            metrics=["mse"],)

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
6809/6809 [==============================] - 302s 43ms/step - loss: 1.2309 - mse: 0.1628 - val_loss: 1.1862 - val_mse: 0.1290
Epoch 2/10
6809/6809 [==============================] - 294s 43ms/step - loss: 1.1872 - mse: 0.1307 - val_loss: 1.1831 - val_mse: 0.1268
Epoch 3/10
6809/6809 [==============================] - 294s 43ms/step - loss: 1.1821 - mse: 0.1259 - val_loss: 1.1923 - val_mse: 0.1360
Epoch 4/10
6809/6809 [==============================] - 289s 42ms/step - loss: 1.1788 - mse: 0.1227 - val_loss: 1.1790 - val_mse: 0.1228
Epoch 5/10
6809/6809 [==============================] - 293s 43ms/step - loss: 1.1763 - mse: 0.1202 - val_loss: 1.1746 - val_mse: 0.1182
Epoch 6/10
6809/6809 [==============================] - 291s 43ms/step - loss: 1.1748 - mse: 0.1187 - val_loss: 1.1789 - val_mse: 0.1224
Epoch 7/10
6809/6809 [==============================] - 292s 43ms/step - loss: 1.1739 - mse: 0.1178 - val_loss: 1.2471 - val_mse: 0.1871
Epoch 8/10
6809/6809 [===================

## Validating model

In [45]:
true_valences, pred_valences, true_arousals, pred_arousals = [], [], [], []

for batch in test_dataset.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index][0])
    true_arousals.append(batch[1][index][1])
    pred_valences.append(batch_predictions[index][0])
    pred_arousals.append(batch_predictions[index][1])
print('collected data for metrics')

1/1 [==============================] - 0s 36ms/step
collected data for metrics


In [46]:
import sklearn.metrics as sm

print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

print('\nArousal metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_arousals, pred_arousals), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_arousals, pred_arousals), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_arousals, pred_arousals), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_arousals, pred_arousals), 2))
print("R2 score =", round(sm.r2_score(true_arousals, pred_arousals), 2))

Valence metrics:
Mean absolute error = 0.29
Mean squared error = 0.15
Median absolute error = 0.21
Explain variance score = 0.45
R2 score = 0.44

Arousal metrics:
Mean absolute error = 0.21
Mean squared error = 0.08
Median absolute error = 0.14
Explain variance score = 0.1
R2 score = 0.08


# Version 2.1 - the same data, increasing model depth to check its performance with more neurons

Low variance and R2 scores demonstrated lack of model efficiency. Considering the values it is innaceptable, but the result can be improved via checking several moments:

1. Making NN deeper, giving more neurons for feature extraction and interpretation, calculating more data for efficient regression;
2. Change loss function and reduce step
2. Perform deeper convolutions over the data, checking different combinations;
3. Use of the leaky RELU instead of simple RELU to avoid dying RELU problem.

Current version checks deeper model. There is no need in detailed explanation.

## Forming NN

In [47]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, first_degree)
x = conv_bn(x, third_degree)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.4)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(2, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_22 (Conv1D)             (None, 468, 32)      128         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_34 (BatchN  (None, 468, 32)     128         ['conv1d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 468, 32)      0           ['batch_normalization

## Training NN

In [48]:
model.compile(loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.002),
            metrics=["mse"],)

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
6809/6809 [==============================] - 444s 62ms/step - loss: 2.2604 - mse: 0.1638 - val_loss: 2.2668 - val_mse: 0.1599
Epoch 2/10
6809/6809 [==============================] - 415s 61ms/step - loss: 2.2053 - mse: 0.1325 - val_loss: 2.2268 - val_mse: 0.1538
Epoch 3/10
6809/6809 [==============================] - 414s 61ms/step - loss: 2.1982 - mse: 0.1260 - val_loss: 2.1985 - val_mse: 0.1259
Epoch 4/10
6809/6809 [==============================] - 395s 58ms/step - loss: 2.1941 - mse: 0.1220 - val_loss: 2.2286 - val_mse: 0.1559
Epoch 5/10
6809/6809 [==============================] - 393s 58ms/step - loss: 2.1915 - mse: 0.1194 - val_loss: 2.2045 - val_mse: 0.1308
Epoch 6/10
6809/6809 [==============================] - 393s 58ms/step - loss: 2.1897 - mse: 0.1177 - val_loss: 2.2134 - val_mse: 0.1403
Epoch 7/10
6809/6809 [==============================] - 393s 58ms/step - loss: 2.1883 - mse: 0.1163 - val_loss: 2.2024 - val_mse: 0.1301
Epoch 8/10
6809/6809 [===================

## Validating model

In [49]:
true_valences, pred_valences, true_arousals, pred_arousals = [], [], [], []

for batch in test_dataset.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index][0])
    true_arousals.append(batch[1][index][1])
    pred_valences.append(batch_predictions[index][0])
    pred_arousals.append(batch_predictions[index][1])
print('collected data for metrics')

1/1 [==============================] - 1s 789ms/step
collected data for metrics


In [50]:
print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

print('\nArousal metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_arousals, pred_arousals), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_arousals, pred_arousals), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_arousals, pred_arousals), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_arousals, pred_arousals), 2))
print("R2 score =", round(sm.r2_score(true_arousals, pred_arousals), 2))

Valence metrics:
Mean absolute error = 0.3
Mean squared error = 0.15
Median absolute error = 0.24
Explain variance score = 0.46
R2 score = 0.44

Arousal metrics:
Mean absolute error = 0.21
Mean squared error = 0.08
Median absolute error = 0.14
Explain variance score = 0.12
R2 score = 0.12


# Version 2.1.1 - the same data, changing loss function and reducing learning rate

In [51]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, first_degree)
x = conv_bn(x, third_degree)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.4)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(2, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_33 (Conv1D)             (None, 468, 32)      128         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_51 (BatchN  (None, 468, 32)     128         ['conv1d_33[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_51 (Activation)     (None, 468, 32)      0           ['batch_normalization

In [53]:
model.compile(loss="mean_absolute_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
6809/6809 [==============================] - 415s 59ms/step - loss: 3.0530 - mse: 0.2034 - mae: 0.3334 - val_loss: 2.7297 - val_mse: 2.3433 - val_mae: 0.5215
Epoch 2/10
6809/6809 [==============================] - 397s 58ms/step - loss: 2.3869 - mse: 0.1570 - mae: 0.2862 - val_loss: 2.3683 - val_mse: 0.1720 - val_mae: 0.2843
Epoch 3/10
6809/6809 [==============================] - 401s 59ms/step - loss: 2.3436 - mse: 0.1423 - mae: 0.2674 - val_loss: 2.3448 - val_mse: 0.1380 - val_mae: 0.2675
Epoch 4/10
6809/6809 [==============================] - 408s 60ms/step - loss: 2.4518 - mse: 0.1482 - mae: 0.2758 - val_loss: 2.4658 - val_mse: 0.1626 - val_mae: 0.2842
Epoch 5/10
6809/6809 [==============================] - 405s 59ms/step - loss: 2.3354 - mse: 0.1366 - mae: 0.2615 - val_loss: 2.3275 - val_mse: 0.1317 - val_mae: 0.2549
Epoch 6/10
6809/6809 [==============================] - 406s 60ms/step - loss: 2.3319 - mse: 0.1344 - mae: 0.2588 - val_loss: 2.3308 - val_mse: 0.1378 - va

In [54]:
print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

print('\nArousal metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_arousals, pred_arousals), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_arousals, pred_arousals), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_arousals, pred_arousals), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_arousals, pred_arousals), 2))
print("R2 score =", round(sm.r2_score(true_arousals, pred_arousals), 2))

Valence metrics:
Mean absolute error = 0.3
Mean squared error = 0.15
Median absolute error = 0.24
Explain variance score = 0.46
R2 score = 0.44

Arousal metrics:
Mean absolute error = 0.21
Mean squared error = 0.08
Median absolute error = 0.14
Explain variance score = 0.12
R2 score = 0.12


## Checking for less epoches

In [56]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree)
x = conv_bn(x, 64)
x = conv_bn(x, first_degree)
x = conv_bn(x, third_degree)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree)
x = layers.Dropout(0.4)(x)
x = dense_bn(x, first_degree)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(2, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d_44 (Conv1D)             (None, 468, 32)      128         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_68 (BatchN  (None, 468, 32)     128         ['conv1d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_68 (Activation)     (None, 468, 32)      0           ['batch_normalization

In [57]:
model.compile(loss="mean_absolute_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_dataset, epochs=8, validation_data=test_dataset)

Epoch 1/8
6809/6809 [==============================] - 447s 63ms/step - loss: 2.3896 - mse: 0.1959 - mae: 0.2991 - val_loss: 2.3668 - val_mse: 0.2474 - val_mae: 0.2854
Epoch 2/8
6809/6809 [==============================] - 410s 60ms/step - loss: 2.3371 - mse: 0.1394 - mae: 0.2638 - val_loss: 2.3689 - val_mse: 0.1568 - val_mae: 0.2737
Epoch 3/8
6809/6809 [==============================] - 410s 60ms/step - loss: 2.3604 - mse: 0.1501 - mae: 0.2762 - val_loss: 2.5421 - val_mse: 2.3893 - val_mae: 0.4658
Epoch 4/8
6809/6809 [==============================] - 410s 60ms/step - loss: 2.3332 - mse: 0.1368 - mae: 0.2604 - val_loss: 2.3260 - val_mse: 0.1331 - val_mae: 0.2538
Epoch 5/8
6809/6809 [==============================] - 416s 61ms/step - loss: 2.3312 - mse: 0.1351 - mae: 0.2588 - val_loss: 2.3520 - val_mse: 0.1401 - val_mae: 0.2677
Epoch 6/8
6809/6809 [==============================] - 406s 60ms/step - loss: 2.3286 - mse: 0.1324 - mae: 0.2564 - val_loss: 2.3413 - val_mse: 0.1320 - val_mae:

In [58]:
print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

print('\nArousal metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_arousals, pred_arousals), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_arousals, pred_arousals), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_arousals, pred_arousals), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_arousals, pred_arousals), 2))
print("R2 score =", round(sm.r2_score(true_arousals, pred_arousals), 2))

Valence metrics:
Mean absolute error = 0.3
Mean squared error = 0.15
Median absolute error = 0.24
Explain variance score = 0.46
R2 score = 0.44

Arousal metrics:
Mean absolute error = 0.21
Mean squared error = 0.08
Median absolute error = 0.14
Explain variance score = 0.12
R2 score = 0.12


# Version 2.2 - the same data, check of the leaky-RELU approach, reduce of dropout to 1/4

There is one drawback of the RELU approach - it is able to cause dead neurons, when there is updated such a weight that neuron will never be activated. So, while simple RELU works with principle of max(0, x) will be used another concept of max(0.1x, x), but in case of the left side will be applied "negative" side to update weights considering "negative" experience. Taking into account that application of the leaky RELU leads to avoiding of 'dead' (or 'deadly overfitted') neurons it is possible to reduce dropout from 0.4 to 0.25

In [4]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, 'leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = conv_bn(x, first_degree, activation_function='leaky_relu')
x = conv_bn(x, third_degree, activation_function='leaky_relu')
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(2, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 468, 32)      0           ['batch_normalization

In [10]:
model.compile(loss="mean_absolute_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_dataset, epochs=8, validation_data=test_dataset)

Epoch 1/8
6809/6809 [==============================] - 420s 59ms/step - loss: 2.3895 - mse: 0.1806 - mae: 0.2931 - val_loss: 2.3366 - val_mse: 0.1402 - val_mae: 0.2618
Epoch 2/8
6809/6809 [==============================] - 403s 59ms/step - loss: 2.3355 - mse: 0.1382 - mae: 0.2629 - val_loss: 2.3511 - val_mse: 0.1671 - val_mae: 0.2790
Epoch 3/8
6809/6809 [==============================] - 394s 58ms/step - loss: 2.3308 - mse: 0.1338 - mae: 0.2585 - val_loss: 2.3350 - val_mse: 0.1460 - val_mae: 0.2626
Epoch 4/8
6809/6809 [==============================] - 393s 58ms/step - loss: 2.3258 - mse: 0.1287 - mae: 0.2535 - val_loss: 2.3401 - val_mse: 0.1334 - val_mae: 0.2676
Epoch 5/8
6809/6809 [==============================] - 393s 58ms/step - loss: 2.3238 - mse: 0.1271 - mae: 0.2516 - val_loss: 2.3237 - val_mse: 0.1292 - val_mae: 0.2514
Epoch 6/8
6809/6809 [==============================] - 394s 58ms/step - loss: 2.3224 - mse: 0.1259 - mae: 0.2502 - val_loss: 2.3229 - val_mse: 0.1254 - val_mae:

In [13]:
true_valences, pred_valences, true_arousals, pred_arousals = [], [], [], []

for batch in test_dataset.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index][0])
    true_arousals.append(batch[1][index][1])
    pred_valences.append(batch_predictions[index][0])
    pred_arousals.append(batch_predictions[index][1])
print('collected data for metrics')

1/1 [==============================] - 1s 821ms/step
collected data for metrics


In [14]:
import sklearn.metrics as sm

print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))

print('\nArousal metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_arousals, pred_arousals), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_arousals, pred_arousals), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_arousals, pred_arousals), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_arousals, pred_arousals), 2))
print("R2 score =", round(sm.r2_score(true_arousals, pred_arousals), 2))

Valence metrics:
Mean absolute error = 0.34
Mean squared error = 0.23
Median absolute error = 0.21
Explain variance score = 0.28
R2 score = 0.14

Arousal metrics:
Mean absolute error = 0.21
Mean squared error = 0.14
Median absolute error = 0.12
Explain variance score = -0.52
R2 score = -0.52


# Version 2.3 - verification of making 2 separate NNs for arousal and valence

Checking suggestion that there is a problem with optimizing model for working with both valence and arousal. In case of getting higher R2 and explaining variance score theory will be proven

## Reading the data and forming TF slices

In [2]:
train_df = read_all_pickles(package_name='pkls_valence_arousal', file_specification="_landmarks_valence_arousal.pkl")
print('original length = ' + str(len(train_df)))

#   drop empty records
train_df.dropna(inplace=True)

#   ignoring the last three labels because they are not providing necessary information
train_df = train_df[(train_df.emotion != 8) & (train_df.emotion != 9) & (train_df.emotion != 10)]
train_df.drop(columns=['emotion'], inplace=True)

print('updated length = ' + str(len(train_df)))
train_df.head(10)

original length = 409797
updated length = 272340


,path,valence,arousal,landmarks
400002,Manually_Annotated_Images/325/f489e8169ea30897...,0.577985,0.271739,"[[0.5394599437713623, 0.676323413848877, -0.08..."
400003,Manually_Annotated_Images/264/224ee36d52b497b1...,-0.384534,0.110619,"[[0.5218021273612976, 0.7440579533576965, -0.1..."
400005,Manually_Annotated_Images/980/8262220f6a3bcfd7...,0.639259,0.034451,"[[0.5117629766464233, 0.7039216160774231, -0.0..."
400007,Manually_Annotated_Images/609/5b3d3b8b9d89deaf...,0.705858,-0.258112,"[[0.47549158334732056, 0.7176483273506165, -0...."
400009,Manually_Annotated_Images/958/7b235cdca2ca0430...,-0.103175,-0.341270,"[[0.5392422080039978, 0.7580064535140991, -0.0..."
400013,Manually_Annotated_Images/715/bff238e798b70ccf...,-0.595238,0.428571,"[[0.49899062514305115, 0.7014129161834717, -0...."
400014,Manually_Annotated_Images/514/f92fad99d07e4b9e...,-0.101626,0.895277,"[[0.47227370738983154, 0.6721517443656921, -0...."
400015,Manually_Annotated_Images/728/9629f4764d5956d9...,0.526225,-0.081953,"[[0.519411563873291, 0.739055871963501, -0.060..."
400016,Manually_Annotated_Images/866/ec55abeb8a12d104...,0.871080,0.183895,"[[0.5619162917137146, 0.6456629633903503, -0.0..."
400017,Manually_Annotated_Images/907/7afaf57f140a6270...,0.691384,-0.062657,"[[0.5264552235603333, 0.7333130240440369, -0.0..."


In [3]:
X = np.array(list(map(np.float64, np.array(train_df['landmarks']))))
Y_valence = np.array(train_df['valence'])

x_train, x_test, y_train_valence, y_test_valence = train_test_split(X, Y_valence, test_size=0.2, random_state=42)

In [4]:
#   to evade memory overflow it is required to remove unnecessary data
del X
del Y_valence
del train_df
gc.collect()

22

In [5]:
train_valence_tf_df = tf.data.Dataset.from_tensor_slices((x_train, y_train_valence))
test_valence_tf_df = tf.data.Dataset.from_tensor_slices((x_test, y_test_valence))

train_valence_tf_df = train_valence_tf_df.shuffle(len(x_train)).batch(32)
test_valence_tf_df = test_valence_tf_df.shuffle(len(x_train)).batch(32)

In [6]:
del y_train_valence
del y_test_valence
gc.collect()

del x_train
del x_test
gc.collect()

0

## Forming NN

In [7]:
first_degree, second_degree, third_degree, fourth_degree = 128, 256, 512, 1024

inputs = keras.Input(shape=(468, 3))

x = tnet(inputs, 3, first_degree, second_degree, third_degree, fourth_degree, 'leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = tnet(x, 64, first_degree, second_degree, third_degree, fourth_degree, activation_function='leaky_relu')
x = conv_bn(x, 64, activation_function='leaky_relu')
x = conv_bn(x, first_degree, activation_function='leaky_relu')
x = conv_bn(x, third_degree, activation_function='leaky_relu')
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, second_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)
x = dense_bn(x, first_degree, activation_function='leaky_relu')
x = layers.Dropout(0.25)(x)

outputs = layers.Dense(1, activation="linear")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet_regr")
model.summary()

Model: "pointnet_regr"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 468, 3)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 468, 32)      128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 468, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 468, 32)      0           ['batch_normalization

## Training NN for valence

In [8]:
model.compile(loss="mean_absolute_error",
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            metrics=["mse", 'mae'],)

model.fit(train_valence_tf_df, epochs=8, validation_data=test_valence_tf_df)

Epoch 1/8
6809/6809 [==============================] - 397s 57ms/step - loss: 2.4509 - mse: 0.2278 - mae: 0.3481 - val_loss: 2.4412 - val_mse: 0.2510 - val_mae: 0.3640
Epoch 2/8
6809/6809 [==============================] - 400s 59ms/step - loss: 2.3821 - mse: 0.1780 - mae: 0.3093 - val_loss: 2.3807 - val_mse: 0.1833 - val_mae: 0.3081
Epoch 3/8
6809/6809 [==============================] - 403s 59ms/step - loss: 2.3751 - mse: 0.1712 - mae: 0.3027 - val_loss: 2.3779 - val_mse: 0.1887 - val_mae: 0.3056
Epoch 4/8
6809/6809 [==============================] - 398s 58ms/step - loss: 2.3725 - mse: 0.1691 - mae: 0.3002 - val_loss: 2.3847 - val_mse: 0.1684 - val_mae: 0.3123
Epoch 5/8
6809/6809 [==============================] - 397s 58ms/step - loss: 2.3698 - mse: 0.1665 - mae: 0.2975 - val_loss: 2.3633 - val_mse: 0.1608 - val_mae: 0.2910
Epoch 6/8
6809/6809 [==============================] - 397s 58ms/step - loss: 2.3666 - mse: 0.1639 - mae: 0.2943 - val_loss: 2.3620 - val_mse: 0.1636 - val_mae:

## Validating valence model

In [9]:
true_valences, pred_valences = [], []

for batch in test_valence_tf_df.as_numpy_iterator():
  batch_predictions = model.predict(batch[0])
  for index in range(len(batch_predictions)):
    true_valences.append(batch[1][index])
    pred_valences.append(batch_predictions[index])
print('collected data for metrics')

1/1 [==============================] - 1s 721ms/step
collected data for metrics


In [ ]:
import sklearn.metrics as sm

print('Valence metrics:')

print("Mean absolute error =", round(sm.mean_absolute_error(true_valences, pred_valences), 2))
print("Mean squared error =", round(sm.mean_squared_error(true_valences, pred_valences), 2))
print("Median absolute error =", round(sm.median_absolute_error(true_valences, pred_valences), 2))
print("Explain variance score =", round(sm.explained_variance_score(true_valences, pred_valences), 2))
print("R2 score =", round(sm.r2_score(true_valences, pred_valences), 2))